<a href="https://colab.research.google.com/github/youkiti/oyakudachi/blob/main/%E3%80%90%E7%A7%91%E7%A0%94%E8%B2%BB%E7%94%A8%E3%80%91%E7%A0%94%E7%A9%B6%E3%83%87%E3%83%BC%E3%82%BF%E3%83%9E%E3%83%8D%E3%82%B8%E3%83%A1%E3%83%B3%E3%83%88%E3%83%97%E3%83%A9%E3%83%B3%EF%BC%88DMP%EF%BC%89%E4%B8%8B%E6%9B%B8%E3%81%8D%E3%82%B3%E3%83%BC%E3%83%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 科研費における研究データの管理・利活用についてに基づくDMPを下書きしてくれるアプリです
## 参考
## https://www.jsps.go.jp/j-grantsinaid/01_seido/10_datamanagement/index.html

#必要なもの
* 研究課題番号
* Claude 3のAPI (最初は$5無料で提供されるので、このシステムを1回まわすくらいなら問題ありません)
##### (公式) https://www.anthropic.com/api
##### (設定方法) https://apidog.com/jp/blog/claude-3-api-tutorial/

#データセット情報の生成もやってくれるようにしていますが、くれぐれもそのまま提出しないでください

In [ ]:
# 研究課題番号を入力する
research_id = input("研究課題番号を入力してください: ")


In [2]:
import requests
from bs4 import BeautifulSoup

def get_kaken_info(research_id):
    url = f"https://kaken.nii.ac.jp/ja/grant/KAKENHI-PROJECT-{research_id}/"
    response = requests.get(url)
    html_content = response.text

    # HTMLのパース
    soup = BeautifulSoup(html_content, 'html.parser')

    # title
    title = soup.find(class_='page-title project').get_text(strip=True)

    # 研究代表者情報の抽出　変数に格納
    chief_investigator_elem = soup.select_one('tr:has(> th:-soup-contains("研究代表者"))')
    chief_investigator_name = ""
    chief_investigator_affiliation = ""
    chief_investigator_number = ""
    if chief_investigator_elem:
        chief_investigator_name = chief_investigator_elem.select_one('td > h4 > span > a').get_text(strip=True)
        chief_investigator_affiliation = chief_investigator_elem.select_one('td > h4').get_text(strip=True)
        chief_investigator_affiliation = chief_investigator_affiliation.replace(chief_investigator_name, '').replace(',\xa0', ' ').strip()
        chief_investigator_number = chief_investigator_affiliation.split('(')[-1].rstrip(')')
        chief_investigator_affiliation = chief_investigator_affiliation.rstrip(f' ({chief_investigator_number})')

    # 研究分担者情報の抽出
    investigator_elem = soup.find('th', string=lambda text: '研究分担者' in text)
    investigator_names = []
    investigator_affiliations = []
    investigator_numbers = []

    # 研究分担者情報の抽出
    investigator_elem = soup.find('th', string=lambda text: '研究分担者' in text)
    investigator_names = []
    investigator_affiliations = []
    investigator_numbers = []

    if investigator_elem:
        investigator_td = investigator_elem.find_next_sibling('td')
        investigator_links = investigator_td.find_all('a')

        for link in investigator_links:
            investigator_name = link.text.strip()
            investigator_names.append(investigator_name)

            affiliation_number = link.next_sibling.strip()
            affiliation, number = affiliation_number.rsplit('(', 1)
            affiliation = affiliation.replace(',\xa0', ' ').strip()
            investigator_affiliations.append(affiliation.strip())
            investigator_numbers.append(number.rstrip(')'))

    # 研究開始時の研究の概要の抽出
    research_overview_elem = soup.select_one('tr:has(> th:-soup-contains("研究開始時の研究の概要"))')
    research_overview = ""
    if research_overview_elem:
        research_overview = research_overview_elem.select_one('td').get_text(strip=True)

    # 研究実績の概要の抽出
    research_achievements_elem = soup.select_one('tr:has(> th:-soup-contains("研究実績の概要"))')
    research_achievements = ""
    if research_achievements_elem:
        research_achievements = research_achievements_elem.select_one('td').get_text(strip=True)

    # 現在までの達成度 (区分)の抽出
    current_progress_elem = soup.select_one('tr:has(> th:-soup-contains("現在までの達成度 (区分)"))')
    current_progress = ""
    if current_progress_elem:
        current_progress = current_progress_elem.select_one('td > p').get_text(strip=True)

    # 今後の研究の推進方策の抽出
    future_plans_elem = soup.select_one('tr:has(> th:-soup-contains("今後の研究の推進方策"))')
    future_plans = ""
    if future_plans_elem:
        future_plans = future_plans_elem.select_one('td').get_text(strip=True)

    return {
        "title": title,
        "chief_investigator_name": chief_investigator_name,
        "chief_investigator_affiliation": chief_investigator_affiliation,
        "chief_investigator_number": chief_investigator_number,
        "investigator_names": investigator_names,
        "investigator_affiliations": investigator_affiliations,
        "investigator_numbers": investigator_numbers,
        "research_overview": research_overview,
        "research_achievements": research_achievements,
        "current_progress": current_progress,
        "future_plans": future_plans
    }

#値を取得
page_info = get_kaken_info(research_id)



#claude 3のセットアップ


In [4]:
!pip install -q python-docx anthropic

import os
import anthropic
import json
from google.colab import userdata
CLAUDE_API_KEY = userdata.get('CLAUDE_API_KEY')
os.environ["CLAUDE_API_KEY"] = CLAUDE_API_KEY
client = anthropic.Anthropic(api_key=os.environ["CLAUDE_API_KEY"])

In [ ]:
# 想定されるデータセットの生成
def generate_research_data_info(research_overview, research_achievements, current_progress,future_plans):
    query = f"""
    以下の研究概要、研究実績、現在までの達成度、今後の研究の推進方策を元に、データごとのデータマネージメントプラン情報をJSONフォーマットで出力してください。

    研究開始時の研究の概要:
    {research_overview}

    研究実績の概要:
    {research_achievements}

    現在までの達成度 (区分):
    {current_progress}

    今後の研究の推進方策：
    {future_plans}
    """

    tools = [
    {
        "name": "output_research_data_info",
        "description": "Output the research data information in JSON format for multiple data sets",
        "input_schema": {
            "type": "object",
            "properties": {
                "data_sets": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "data_name": {
                                "type": "string",
                                "description": "Name of the research data",
                            },
                            "data_overview": {
                                "type": "string",
                                "description": "Overview of the research data",
                            },
                            "data_collector": {
                                "type": "string",
                                "description": "Collector or obtainer of the research data",
                            },
                            "data_manager": {
                                "type": "string",
                                "description": "Manager of the research data (only if different from the collector or obtainer)",
                            },
                            "sensitive_info_policy": {
                                "type": "string",
                                "description": "機微情報がある場合の取り扱い方針. ",
                            },
                            "data_sharing_policy": {
                                "type": "string",
                                "description": "研究データの公開・提供方針。公開・共有・非公開非共有のいずれか",
                            },
                            "data_sharing_details": {
                                "type": "string",
                                "description": "研究データの公開・提供方針詳細",
                            },
                        },
                        "required": [
                            "data_name",
                            "data_overview",
                            "data_collector",
                            "sensitive_info_policy",
                            "data_sharing_policy",
                            "data_sharing_details",
                        ],
                    }
                }
            },
            "required": ["data_sets"],
        }
    }

    ]

    response = client.beta.tools.messages.create(
        model="claude-3-opus-20240229",
        system = "あなたは優秀な研究アシスタントです。与えられた情報から考えられる限りの研究データ（個別の患者データ、実験データ、解析コードなど）の適切な研究データマネジメントのための情報を日本語で考えられるだけ生成してください。",
        messages=[
            {"role": "user", "content": query}
        ],
        max_tokens=4096,
        tools=tools,
        temperature=0.3 #これが最適かは不明
    )

    message = response.content[1].input
    return message

#データ生成
research_data_info = generate_research_data_info(
    page_info["research_overview"],
    page_info["research_achievements"],
    page_info["current_progress"],
    page_info["future_plans"]
)


#Wordファイルにしてダウンロード

In [7]:
from google.colab import files
from datetime import date
from docx import Document
from docx.shared import Pt
from docx.enum.style import WD_STYLE_TYPE
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.section import WD_ORIENT

# Wordドキュメントの作成
doc = Document()

# スタイルの設定
style = doc.styles.add_style('Table Cell', WD_STYLE_TYPE.CHARACTER)
font = style.font
font.name = 'メイリオ'
font.size = Pt(11)

# 今日の日付を取得
today = date.today().strftime("%Y年%m月%d日")

#title
title = doc.add_heading(f"{page_info['title']} データマネジメントプラン", level=1)
title.alignment = WD_ALIGN_PARAGRAPH.CENTER


# 1. DMP作成・更新情報
doc.add_heading("1. DMP作成・更新情報", level=1)
table1 = doc.add_table(rows=2, cols=2)
table1.style = 'Table Grid'

cell = table1.cell(0, 0)
cell.text = "DMP作成年月日"
cell.paragraphs[0].runs[0].style = 'Table Cell'

cell = table1.cell(0, 1)
cell.text = today
cell.paragraphs[0].runs[0].style = 'Table Cell'

cell = table1.cell(1, 0)
cell.text = "DMP最終更新年月日"
cell.paragraphs[0].runs[0].style = 'Table Cell'

cell = table1.cell(1, 1)
cell.text = today
cell.paragraphs[0].runs[0].style = 'Table Cell'

# 2. 研究課題情報
doc.add_heading("2. 研究課題情報", level=1)
table2 = doc.add_table(rows=1, cols=2)
table2.style = 'Table Grid'

cell = table2.cell(0, 0)
cell.text = "研究課題番号"
cell.paragraphs[0].runs[0].style = 'Table Cell'

cell = table2.cell(0, 1)
cell.text = research_id
cell.paragraphs[0].runs[0].style = 'Table Cell'

# 3. 担当者情報
doc.add_heading("3. 担当者情報", level=1)

# 表の作成
table2 = doc.add_table(rows=4+len(page_info["investigator_names"]), cols=6)
table2.style = 'Table Grid'

# ヘッダー行の設定
header_row = table2.rows[0]
header_row.cells[0].text = ""
header_row.cells[1].text = "本計画書内通し番号"
header_row.cells[2].text = "氏名"
header_row.cells[3].text = "所属・役職"
header_row.cells[4].text = "研究者番号 ※該当がない場合は空欄可"
header_row.cells[5].text = "連絡先"

# 研究代表者の情報を追加
row = table2.rows[1]
row.cells[0].text = "研究代表者"
row.cells[1].text = "1"
row.cells[2].text = page_info["chief_investigator_name"]
row.cells[3].text = page_info["chief_investigator_affiliation"]
row.cells[4].text = page_info["chief_investigator_number"]

# 研究分担者の情報を追加
if page_info["investigator_names"]:
    for i in range(len(page_info["investigator_names"])):
        row = table2.rows[i+2]
        row.cells[0].text = "研究分担者"
        row.cells[1].text = str(i+2)
        row.cells[2].text = page_info["investigator_names"][i]
        row.cells[3].text = page_info["investigator_affiliations"][i]
        row.cells[4].text = page_info["investigator_numbers"][i]
else:
    # 研究分担者がいない場合、表の行数を調整
    for i in range(len(table2.rows)-5, 0, -1):
        table2._element.remove(table2.rows[i]._element)

# 研究データの取得者又は収集者、管理責任者の行を追加
row = table2.rows[-2]
row.cells[0].text = "研究データの取得者又は収集者"

row = table2.rows[-1]
row.cells[0].text = "研究開発データの管理責任者"

# セルのスタイルを設定
for row in table2.rows:
    for cell in row.cells:
        for paragraph in cell.paragraphs:
            for run in paragraph.runs:
                run.style = 'Table Cell'

# 4. 担当者情報
doc.add_heading("4. 研究データ情報", level=1)

# テーブル作成
# 表の作成
table3 = doc.add_table(rows=1+len(research_data_info['data_sets']), cols=10)
table3.style = 'Table Grid'

# 列名はこの10　No.	研究データの名称	研究データの概要	研究データの取得者又は収集者	"研究データの管理者 ※取得者又は収集者と異なる場合のみ記入"	機微情報がある場合の取り扱い方針	研究データの公開・提供方針	研究データの公開・提供方針詳細	"研究データの公開・提供場所 （URL、DOI）"	研究データ公開日（予定日）

# ヘッダー行の設定
header_row = table3.rows[0]
header_row.cells[0].text = "No."
header_row.cells[1].text = "研究データの名称"
header_row.cells[2].text = "研究データの概要"
header_row.cells[3].text = "研究データの取得者又は収集者"
header_row.cells[4].text = "研究データの管理者 ※取得者又は収集者と異なる場合のみ記入"
header_row.cells[5].text = "機微情報がある場合の取り扱い方針"
header_row.cells[6].text = "研究データの公開・提供方針"
header_row.cells[7].text = "研究データの公開・提供方針詳細"
header_row.cells[8].text = "研究データの公開・提供場所（URL、DOI）"
header_row.cells[9].text = "研究データ公開日（予定日）"

# データ行の設定
for i, data_set in enumerate(research_data_info['data_sets'], start=1):
    row = table3.rows[i]
    row.cells[0].text = str(i)
    row.cells[1].text = data_set.get('data_name', '')
    row.cells[2].text = data_set.get('data_overview', '')
    row.cells[3].text = data_set.get('data_collector', '')
    row.cells[4].text = data_set.get('data_manager', '')
    row.cells[5].text = data_set.get('sensitive_info_policy', '')
    row.cells[6].text = data_set.get('data_sharing_policy', '')
    row.cells[7].text = data_set.get('data_sharing_details', '')

# ファイルの保存
output_filename = f'DMP-{research_id}.docx'
doc.save(output_filename)

# ダウンロード
files.download(output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>